In [1]:
wd <- dirname(dirname(getwd()))
source(paste0(wd,"/mission_control/treasure_map.R"))
source(paste0(wd,"/mission_control/helpers/xgboost/eval_help.R"))
source(paste0(wd,"/mission_control/helpers/xgboost/paper_settings.R"))

library(xgboost)
library(tidyverse)
library(survival)
library(pROC)
library(caret)
library(survcomp)

Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ──────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.2     ✔ dplyr   1.0.6
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.1
Warning message:
“package ‘tibble’ was built under R version 3.6.3”Warning message:
“package ‘tidyr’ was built under R version 3.6.3”Warning message:
“package ‘purrr’ was built under R version 3.6.3”Warning message:
“package ‘dplyr’ was built under R version 3.6.3”Warning message:
“package ‘forcats’ was built under R version 3.6.3”── Conflicts ─────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ dplyr::slice()  masks xgboost::slice()
Type 'cita

### 0 - Read in prepped data

In [2]:
eval_ready <- readRDS(paste0(TMP_DIR, "xg-eval-prep.Rds"))

#### Settings for eval

In [13]:
args <- commandArgs(trailing = TRUE) ##
#args <- list("pfs", "latent_vhio", "true")
model <- args[[1]]
features <- args[[2]]
purity <- args[[3]]
model_features <- unname(unlist(model_features[features]))
if( purity == "true") {
    model_features <- c(model_features, "purity")
}
parameter_grid <- get_parameter_grid(features)

ERROR: Error in args[[2]]: subscript out of bounds


In [4]:
names(model_features)

NULL

In [5]:
print(model); 
print(features);
print(parameter_grid)

[1] "pfs"
[1] "latent_vhio"
   eta max_depth min_child_weight subsample colsample_bytree optimal_trees
1 0.05         1                5      0.75                1             0
  error
1     0


In [6]:
base_tissue <- "all"
N <- 1000
K <- 5

### 1 - Run it

In [7]:
set.seed(622)

In [8]:
evals <- data.frame()

for (j in seq(N)) {
    print(j)
    oo <- evaluate2(  eval_ready, 
                      model = model, 
                      parameter_grid = parameter_grid, 
                      model_features = model_features, 
                      complete = TRUE, 
                      base_tissue = base_tissue, 
                      folds = K ); 

    if (!is.null(oo)){
        eval_i <- data.frame(oo$evals) %>% tibble::rownames_to_column(var = "tissue"); 
        eval_i$model <- model; 
        eval_i$rep <- j; 
        eval_i$complete <- TRUE; 
        eval_i$features <- features
        evals <- rbind(evals, eval_i)
    }
}
evals$base_tissue <- base_tissue
evals$purity <- purity

[1] 1


### 2 - Send it!

In [10]:
saveRDS(evals,paste0(TMP_DIR,paste0( "pred_study/xg-eval-results-",model,"-",features, "-", purity,".Rds" ) ))